# Library Import

In [1]:

# !pip install torch scikit-learn pandas --quiet 

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd


# Functions

In [2]:

# Configuration
NUM_SAMPLES = 1000
SEQ_LEN = 10
NUM_DEPTS = 5
EMBED_DIM = 64
MASK_PROB = 0.15
NUM_SEASONS = 4

def generate_synthetic():
    dept_seq = torch.randint(0, NUM_DEPTS, (NUM_SAMPLES, SEQ_LEN))
    season_ids = torch.randint(0, NUM_SEASONS, (NUM_SAMPLES, SEQ_LEN))

    labels = torch.randint(0, 2, (NUM_SAMPLES, NUM_DEPTS)).float()

    customer_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)
    product_emb = torch.randn(NUM_SAMPLES, SEQ_LEN, EMBED_DIM)
    season_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)
    weather_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)
    promo_emb = torch.randn(NUM_SAMPLES, EMBED_DIM)

    return {
        "sequences": dept_seq,
        "season_ids": season_ids,
        "labels": labels,
        "customer_emb": customer_emb,
        "product_emb": product_emb,
        "season_emb": season_emb,
        "weather_emb": weather_emb,
        "promo_emb": promo_emb
    }

data = generate_synthetic()


In [3]:

def get_decay(seq_len, gamma=0.95):
    decay = torch.tensor([gamma**i for i in reversed(range(seq_len))], dtype=torch.float)
    decay = decay / decay.sum()
    return decay.unsqueeze(0)  # [1, T]


In [4]:

class BERT4Rec(nn.Module):
    def __init__(self, vocab_size, num_depts, emb_dim, seq_len, num_seasons):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size + 1, emb_dim, padding_idx=vocab_size)
        self.pos_emb = nn.Embedding(seq_len, emb_dim)
        self.season_pos_emb = nn.Embedding(num_seasons, emb_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=4, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.struct_proj = nn.Sequential(
            nn.Linear(emb_dim * 4, emb_dim),
            nn.ReLU()
        )

        self.head = nn.Sequential(
            nn.Linear(emb_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, num_depts),
            nn.Sigmoid()
        )

    def forward(self, seqs, season_ids, customer_emb, product_emb, season_emb, weather_emb, promo_emb, decay=None):
        B, T = seqs.shape
        pos_ids = torch.arange(T, device=seqs.device).unsqueeze(0).expand(B, T)

        x = self.token_emb(seqs) + self.season_pos_emb(season_ids) + self.pos_emb(pos_ids)
        x = self.transformer(x)

        if decay is not None:
            x = (x * decay.unsqueeze(-1)).sum(1)  # Weighted sum
        else:
            x = x[:, 0, :]  # CLS token approach

        struct = torch.cat([season_emb, weather_emb, promo_emb, customer_emb], dim=-1)
        struct = self.struct_proj(struct)

        final = torch.cat([x, struct], dim=-1)
        return self.head(final)


In [5]:

def train_model(model, data, epochs=5, batch_size=32, lr=1e-3, gamma=0.95):
    dataset = torch.utils.data.TensorDataset(
        data["sequences"], data["season_ids"], data["labels"],
        data["customer_emb"], data["product_emb"],
        data["season_emb"], data["weather_emb"], data["promo_emb"]
    )
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in loader:
            seqs, season_ids, labels, cust, prod, seas, weath, promo = batch
            decay = get_decay(seqs.size(1)).to(seqs.device)

            out = model(seqs, season_ids, cust, prod, seas, weath, promo, decay)
            loss = loss_fn(out, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")


In [6]:

def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        decay = get_decay(data["sequences"].size(1))
        preds = model(
            data["sequences"], data["season_ids"],
            data["customer_emb"], data["product_emb"],
            data["season_emb"], data["weather_emb"],
            data["promo_emb"], decay
        )
        preds_bin = (preds > 0.5).float()
        f1 = f1_score(data["labels"].numpy(), preds_bin.numpy(), average="macro")
        print(f"Macro F1 score: {f1:.4f}")


# Train Model

In [7]:

model = BERT4Rec(vocab_size=NUM_DEPTS, num_depts=NUM_DEPTS,
                 emb_dim=EMBED_DIM, seq_len=SEQ_LEN, num_seasons=NUM_SEASONS)

train_model(model, data)
evaluate_model(model, data)


Epoch 1: Loss = 22.2748
Epoch 2: Loss = 22.0062
Epoch 3: Loss = 21.6108
Epoch 4: Loss = 20.9157
Epoch 5: Loss = 19.9802
Macro F1 score: 0.6478


# Score Model

In [8]:
def generate_scoring_data():
    # Same schema as training but new samples
    return generate_synthetic()  # reuse your generate_synthetic() function

scoring_data = generate_scoring_data()

In [9]:

def score_and_save(model, scoring_data, save_path="scoring_predictions.csv"):
    model.eval()
    with torch.no_grad():
        decay = get_decay(scoring_data["sequences"].size(1))

        preds = model(
            scoring_data["sequences"], scoring_data["season_ids"],
            scoring_data["customer_emb"], scoring_data["product_emb"],
            scoring_data["season_emb"], scoring_data["weather_emb"],
            scoring_data["promo_emb"], decay
        )

        # Convert predictions to NumPy
        preds_np = preds.cpu().numpy()
        labels_np = scoring_data["labels"].cpu().numpy()

        # Create a DataFrame with columns for each department
        df = pd.DataFrame(preds_np, columns=[f"dept_{i}_score" for i in range(preds_np.shape[1])])
        for i in range(preds_np.shape[1]):
            df[f"dept_{i}_label"] = labels_np[:, i]

        df.to_csv(save_path, index=False)
        print(f"✅ Scoring predictions saved to: {save_path}")
        return df

In [ ]:
# Score new dataset
df = score_and_save(model, scoring_data)